In [11]:
from google.colab import files
files.upload()

Saving k8.json to k8.json


{'k8.json': b'{"username":"maneendrakummari","key":"3ed64ba0c9e295242de6311a68ae976a"}'}

In [12]:
!mv k8.json kaggle.json


In [ ]:
!kaggle datasets download -d lucasheilbuthh/inferring-broiler-chicken-weight
!unzip inferring-broiler-chicken-weight.zip


In [ ]:
import pandas as pd

df = pd.read_csv("broiler_chicken_weight.csv")  # filename may vary
df.head()


In [ ]:
X = df[[df.columns[0]]]   # area feature
y = df[df.columns[1]]     # weight


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

weight_model = LinearRegression()
weight_model.fit(X_train, y_train)

preds = weight_model.predict(X_test)
print("MAE (grams):", mean_absolute_error(y_test, preds))


In [ ]:
import joblib

joblib.dump(weight_model, "weight_model.pkl")
print("Weight model saved")


In [ ]:
#PART 5: INTEGRATE WITH DETECTION + TRACKING OUTPUT
from collections import defaultdict
import numpy as np

bird_area_map = defaultdict(list)

for frame_id, detections in frame_data.items():
    for track_id, area in detections:
        bird_area_map[track_id].append(area)

bird_avg_area = {
    tid: np.mean(areas)
    for tid, areas in bird_area_map.items()
}


In [ ]:
import joblib

weight_model = joblib.load("weight_model.pkl")

bird_weights = {}

for tid, avg_area in bird_avg_area.items():
    est_weight = weight_model.predict([[avg_area]])[0]
    bird_weights[tid] = round(float(est_weight), 2)

bird_weights


In [ ]:
final_output = {
    "bird_count_over_time": bird_count_time_series,
    "estimated_bird_weights": bird_weights
}

final_output


In [ ]:
import json

with open("final_results.json", "w") as f:
    json.dump(final_output, f, indent=4)

print("final_results.json saved")


In [ ]:
#Dummy test prediction
test_area = [[15000]]  # example bbox area
print("Estimated weight:", weight_model.predict(test_area))
